# Part 1 : Data Preparation and Process

### Install required and/or update third-party libraries

In [1]:
!python -m pip install -Uq pip
!python -m pip install -Uq sagemaker boto3

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


### Loading stored variables
If you ran this notebook before, you may want to re-use the resources you aready created with AWS. Run the cell below to load any prevously created variables. You should see a print-out of the existing variables. If you don't see anything printed then it's probably the first time you are running the notebook!

In [4]:
%store -r
%store

Stored variables and their in-db values:


: You must have run the previous sequential notebooks to retrieve variables using the StoreMagic command.

In [5]:
# cell 01
import sagemaker
bucket=sagemaker.Session().default_bucket()
prefix = 'sagemaker/DEMO-xgboost-tripfare'
 
# Define IAM role
import boto3
import re
from sagemaker import get_execution_role

role = get_execution_role()

In [6]:
# cell 02
import numpy as np                                # For matrix operations and numerical processing
import pandas as pd                               # For munging tabular data
import matplotlib.pyplot as plt                   # For charts and visualizations
from IPython.display import Image                 # For displaying images in the notebook
from IPython.display import display               # For displaying outputs in the notebook
from time import gmtime, strftime                 # For labeling SageMaker models, endpoints, etc.
import sys                                        # For writing outputs to notebook
import math                                       # For ceiling function
import json                                       # For parsing hosting outputs
import os                                         # For manipulating filepath names
import sagemaker 
import zipfile     # Amazon SageMaker's Python SDK provides many helper functions

In [7]:
input_source = f's3://{bucket}/{prefix}/input/'
input_source

's3://sagemaker-us-east-1-631450739534/sagemaker/DEMO-xgboost-tripfare/input/'

In [8]:
input_data = input_source + 'data/'
input_zones = input_source + 'zones/'

In [9]:
!aws s3 cp --recursive 's3://nyc-tlc/trip data/' $input_data --exclude '*' --include 'green_tripdata_2018-1*'
# !aws s3 cp 's3://nyc-tlc/trip data/green_tripdata_2018-02.csv' $input_data
!aws s3 cp 's3://nyc-tlc/misc/taxi_zones.zip' $input_zones

copy: s3://nyc-tlc/trip data/green_tripdata_2018-10.csv to s3://sagemaker-us-east-1-631450739534/sagemaker/DEMO-xgboost-tripfare/input/data/green_tripdata_2018-10.csv
copy: s3://nyc-tlc/trip data/green_tripdata_2018-11.csv to s3://sagemaker-us-east-1-631450739534/sagemaker/DEMO-xgboost-tripfare/input/data/green_tripdata_2018-11.csv
copy: s3://nyc-tlc/trip data/green_tripdata_2018-12.csv to s3://sagemaker-us-east-1-631450739534/sagemaker/DEMO-xgboost-tripfare/input/data/green_tripdata_2018-12.csv
copy: s3://nyc-tlc/misc/taxi_zones.zip to s3://sagemaker-us-east-1-631450739534/sagemaker/DEMO-xgboost-tripfare/input/zones/taxi_zones.zip


In [12]:
!aws s3 ls $input_data

2021-09-30 01:59:14   63666398 green_tripdata_2018-10.csv
2021-09-30 01:59:14   58812608 green_tripdata_2018-11.csv
2021-09-30 01:59:14   61371115 green_tripdata_2018-12.csv


In [10]:
# cell 05
from sagemaker import Session

sess = Session()

In [31]:
%%writefile preprocess.py

import glob
import logging
import os
import subprocess
import sys
from zipfile import ZipFile
# from time import gmtime, strftime
import socket
import shutil
import json

host_name = socket.gethostname()
print(host_name)
print(os.environ)

# Install geopandas dependency before including pandas
subprocess.check_call([sys.executable, "-m", "pip", "install", "geopandas==0.9.0"])

import pandas as pd  # noqa: E402
import geopandas as gpd  # noqa: E402
from sklearn.model_selection import train_test_split  # noqa: E402

logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.addHandler(logging.StreamHandler())


def extract_zones(zones_file: str, zones_dir: str):
    logger.info(f"Extracting zone file: {zones_file}")
    with ZipFile(zones_file, "r") as zip:
        zip.extractall(zones_dir)


def load_zones(zones_dir: str):
    logging.info(f"Loading zones from {zones_dir}")
    # Load the shape file and get the geometry and lat/lon
    zone_df = gpd.read_file(os.path.join(zones_dir, "taxi_zones.shp"))
    # Get centroids as EPSG code of 3310 to measure distance
    zone_df["centroid"] = zone_df.geometry.centroid.to_crs(epsg=3310)
    # Convert cordinates to the WSG84 lat/long CRS has a EPSG code of 4326.
    zone_df["latitude"] = zone_df.centroid.to_crs(epsg=4326).x
    zone_df["longitude"] = zone_df.centroid.to_crs(epsg=4326).y
    return zone_df


def load_data(file_list: list):
    # Define dates, and columns to use
    use_cols = [
        "fare_amount",
        "lpep_pickup_datetime",
        "lpep_dropoff_datetime",
        "passenger_count",
        "PULocationID",
        "DOLocationID",
    ]
    # Concat input files with select columns
    dfs = []
    for file in file_list:
        dfs.append(pd.read_csv(file, usecols=use_cols))
    return pd.concat(dfs, ignore_index=True)


def enrich_data(trip_df: pd.DataFrame, zone_df: pd.DataFrame):
    # Join trip DF to zones for poth pickup and drop off locations
    trip_df = gpd.GeoDataFrame(
        trip_df.join(zone_df, on="PULocationID").join(
            zone_df, on="DOLocationID", rsuffix="_DO", lsuffix="_PU"
        )
    )
    trip_df["geo_distance"] = (
        trip_df["centroid_PU"].distance(trip_df["centroid_DO"]) / 1000
    )

    # Add date parts
    trip_df["lpep_pickup_datetime"] = pd.to_datetime(trip_df["lpep_pickup_datetime"])
    trip_df["hour"] = trip_df["lpep_pickup_datetime"].dt.hour
    trip_df["weekday"] = trip_df["lpep_pickup_datetime"].dt.weekday
    trip_df["month"] = trip_df["lpep_pickup_datetime"].dt.month

    # Get calculated duration in minutes
    trip_df["lpep_dropoff_datetime"] = pd.to_datetime(trip_df["lpep_dropoff_datetime"])
    trip_df["duration_minutes"] = (
        trip_df["lpep_dropoff_datetime"] - trip_df["lpep_pickup_datetime"]
    ).dt.seconds / 60

    # Rename and filter cols
    trip_df = trip_df.rename(
        columns={
            "latitude_PU": "pickup_latitude",
            "longitude_PU": "pickup_longitude",
            "latitude_DO": "dropoff_latitude",
            "longitude_DO": "dropoff_longitude",
        }
    )
    return trip_df


def clean_data(trip_df: pd.DataFrame):
    # Remove outliers
    trip_df = trip_df[
        (trip_df.fare_amount > 0)
        & (trip_df.fare_amount < 200)
        & (trip_df.passenger_count > 0)
        & (trip_df.duration_minutes > 0)
        & (trip_df.duration_minutes < 120)
        & (trip_df.geo_distance > 0)
        & (trip_df.geo_distance < 121)
    ].dropna()

    # Filter columns
    cols = [
        "fare_amount",
        "passenger_count",
        "pickup_latitude",
        "pickup_longitude",
        "dropoff_latitude",
        "dropoff_longitude",
        "geo_distance",
        "hour",
        "weekday",
        "month",
    ]
    return trip_df[cols]


def save_files(base_dir: str, data_df: pd.DataFrame, val_size=0.2, test_size=0.05, current_host=None):
        
    logger.info(f"Splitting {len(data_df)} rows of data into train, val, test.")
    train_df, val_df = train_test_split(data_df, test_size=val_size, random_state=42)
    val_df, test_df = train_test_split(val_df, test_size=test_size, random_state=42)

    logger.info(f"Writing out datasets to {base_dir}")
    train_df.to_csv(f"{base_dir}/train/train-{current_host}.csv", header=False, index=False)
    val_df.to_csv(f"{base_dir}/validation/validation-{current_host}.csv", header=False, index=False)

    # Save test data without header
    test_df.to_csv(f"{base_dir}/test/test-{current_host}.csv", header=False, index=False)

    return train_df, val_df, test_df

def _read_json(path):  # type: (str) -> dict
    """Read a JSON file.
    Args:
        path (str): Path to the file.
    Returns:
        (dict[object, object]): A dictionary representation of the JSON file.
    """
    with open(path, "r") as f:
        return json.load(f)

def main(base_dir):
    # Input data files
    input_dir = os.path.join(base_dir, "input/data")
    input_file_list = glob.glob(f"{input_dir}/*.csv")
    logger.info(f"Input file list: {input_file_list}")
    
    config_file_list = glob.glob("/opt/ml/config/*.json")
    logger.info(f"config file list: {config_file_list}")
    if "/opt/ml/config/resourceconfig.json" in config_file_list:
        hosts = _read_json("/opt/ml/config/resourceconfig.json")
        logger.info(hosts)
        current_host = hosts["current_host"]
        logger.info(current_host)
    else:
        current_host = 'algo-0'
        
    if len(input_file_list) == 0:
        raise Exception(f"No input files found in {input_dir}")

    # Input zones file
    zones_dir = os.path.join(base_dir, "input/zones")
    zones_file = os.path.join(zones_dir, "taxi_zones.zip")
    if not os.path.exists(zones_file):
        raise Exception(f"Zones file {zones_file} does not exist")

    # Extract and load taxi zones geopandas dataframe
    extract_zones(zones_file, zones_dir)
    zone_df = load_zones(zones_dir)

    # Load input files
    data_df = load_data(input_file_list)
    data_df = enrich_data(data_df, zone_df)
    data_df = clean_data(data_df)
    return save_files(base_dir, data_df, current_host=current_host)


if __name__ == "__main__":
    logger.info("Starting preprocessing.")
    main("/opt/ml/processing")
    logger.info("Done")


Overwriting preprocess.py


In [29]:
# cell 07
train_path = f"s3://{bucket}/{prefix}/train"
validation_path = f"s3://{bucket}/{prefix}/validation"
test_path = f"s3://{bucket}/{prefix}/test"


In [32]:
# cell 08
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker import get_execution_role


sklearn_processor = SKLearnProcessor(
    framework_version="0.23-1",
    role=get_execution_role(),
    instance_type="ml.m5.large",
    instance_count=2, 
    base_job_name='sm-immday-skprocessing'
)

sklearn_processor.run(
    code='preprocess.py',
    inputs=[
        ProcessingInput(
            source=input_data,
            destination="/opt/ml/processing/input/data",
            s3_data_distribution_type="ShardedByS3Key",
        ),
        ProcessingInput(
            source=input_zones,
            destination="/opt/ml/processing/input/zones",
            s3_data_distribution_type="FullyReplicated",
        ),
    ],
    outputs=[
        ProcessingOutput(output_name="train", source="/opt/ml/processing/train", destination=train_path),
        ProcessingOutput(output_name="validation", source="/opt/ml/processing/validation", destination=validation_path),
        ProcessingOutput(output_name="test", source="/opt/ml/processing/test", destination=test_path),
    ]
        
)


Job Name:  sm-immday-skprocessing-2021-09-30-04-46-49-652
Inputs:  [{'InputName': 'input-1', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-631450739534/sagemaker/DEMO-xgboost-tripfare/input/data/', 'LocalPath': '/opt/ml/processing/input/data', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'ShardedByS3Key', 'S3CompressionType': 'None'}}, {'InputName': 'input-2', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-631450739534/sagemaker/DEMO-xgboost-tripfare/input/zones/', 'LocalPath': '/opt/ml/processing/input/zones', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-631450739534/sm-immday-skprocessing-2021-09-30-04-46-49-652/input/code/preprocess.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistr

In [19]:
%store train_path
%store validation_path
%store test_path

Stored 'train_path' (str)
Stored 'validation_path' (str)
Stored 'test_path' (str)
